In [ ]:
import pandas as pd
import os
import numpy as np
import PIL.Image
import glob

import tensorflow
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import cv2
import tqdm
from IPython.display import Image
%matplotlib inline

os.listdir("../input/cifar-10")

# Importing training labels

In [ ]:
!pip install py7zr

In [ ]:
!python -m py7zr x ../input/cifar-10/train.7z 

In [ ]:
y = pd.read_csv('../input/cifar-10/trainLabels.csv')
file_list = glob.glob("../working/train/*")

In [ ]:
y

## Extracting Image Names

In [ ]:
file_list = sorted(file_list,key= lambda x:int(x.split("/")[-1].split(".")[0]))

## Forming our training X

In [ ]:
X = np.array([np.array(PIL.Image.open(f_name)) for f_name in file_list])
print(f"{X.shape} -- {y.shape}")

In [ ]:
pd.CategoricalIndex(y['label'].values).categories

In [ ]:
temp = y['label'].values
X = X.astype('float32')
#Normalizing
X = X/255.0
categories = list(pd.CategoricalIndex(temp).categories)
mapped = {}
rev_mapped = {}
for i in range(len(categories)):
    mapped[categories[i]] = i
    rev_mapped[i] = categories[i]

for i in range(len(temp)):
    temp[i] = mapped[temp[i]]
    
#y is the one-hot-vector for each datapoint/image
y = tensorflow.keras.utils.to_categorical(temp)

In [ ]:
print(f"{X.shape} -- {y.shape}")

# Model Construction


In [ ]:

model = Sequential()
model.add(Conv2D(24,kernel_size=(3,3),padding='same',activation='relu',input_shape=(32,32,3)))
model.add(BatchNormalization())
model.add(Conv2D(24,kernel_size=(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Conv2D(128,kernel_size=4,activation='relu'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(10,activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])





In [ ]:
model.summary()

# Data Augmentation


In [ ]:
datagen = ImageDataGenerator(rotation_range=10, zoom_range=0.1, width_shift_range=0.1, height_shift_range=0.1)
datagen.fit(X)

## Splitting dataset into training and testing

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=123)

# Training our CNN Model

In [ ]:
history = model.fit(datagen.flow(X_train,y_train),batch_size=128,epochs=40,verbose=2,validation_data=(X_test,y_test))

## We have 85% accuracy with this simple CNN model!!

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs=range(len(acc))

plt.plot(epochs,acc, )
plt.plot(epochs,val_acc)
plt.title('Training and validation accuracy')
plt.legend(["training accuracy", "validation accuracy"], loc ="lower right") 

plt.figure()

plt.xlabel("Epochs")
plt.plot(epochs,loss)
plt.plot(epochs,val_loss)
plt.title('Training and validation loss')
plt.legend(["training loss", "validation loss"], loc ="upper right") 

# Some Missclassifications

In [ ]:
pred = model.predict(np.expand_dims(X_train[10], axis=0))
plt.figure(figsize = (1,1))
plt.imshow(X_train[10])
plt.xlabel(rev_mapped[np.argmax(pred, axis=1)[0]])


In [ ]:
pred = model.predict(np.expand_dims(X_train[1], axis=0))
plt.figure(figsize = (1,1))
plt.imshow(X_train[1])
plt.xlabel(rev_mapped[np.argmax(pred, axis=1)[0]])


## Some Missclassifications